Mini-Project 1 : Scraping Data From A Dynamic Webpage

Scrape data from a dynamic webpage using Selenium and BeautifulSoup
Control a web browser using Selenium to interact with JavaScript-heavy web pages.


In [1]:
!apt-get update
!apt-get install -y chromium-browser

!pip install selenium
!pip install beautifulsoup4
!pip install chromedriver-autoinstaller

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,591 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,442 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:14 https://ppa.launchpadco

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chromedriver_autoinstaller.install()


chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://github.com/topics')

try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'topic-box'))
    )
except Exception as e:
    print("An error occurred while waiting for the page to load:", e)

soup = BeautifulSoup(driver.page_source, 'html.parser')

topic_title_tags = soup.find_all('p', {'class': 'f3 lh-condensed mb-0 mt-1 Link--primary'})
topic_titles = [tag.text.strip() for tag in topic_title_tags]

topic_desc_tags = soup.find_all('p', {'class': 'f5 color-fg-muted mb-0 mt-1'})
topic_descriptions = [tag.text.strip() for tag in topic_desc_tags]

topic_link_tags = soup.find_all('a', {'class': 'no-underline flex-1 d-flex flex-column'})
topic_urls = ['https://github.com' + tag.get('href') for tag in topic_link_tags]

data = {
    'Title': topic_titles,
    'Description': topic_descriptions,
    'URL': topic_urls
}

df = pd.DataFrame(data)

print(df)
driver.quit()

                     Title                                        Description  \
0                       3D  3D refers to the use of three-dimensional grap...   
1                     Ajax  Ajax is a technique for creating interactive w...   
2                Algorithm  Algorithms are self-contained sequences that c...   
3                      Amp  Amp is a non-blocking concurrency library for ...   
4                  Android  Android is an operating system built by Google...   
5                  Angular  Angular is an open source web application plat...   
6                  Ansible  Ansible is a simple and powerful automation en...   
7                      API  An API (Application Programming Interface) is ...   
8                  Arduino  Arduino is an open source platform for buildin...   
9                  ASP.NET  ASP.NET is a web framework for building modern...   
10           Awesome Lists  An awesome list is a list of awesome things cu...   
11     Amazon Web Services  

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchFrameException
from bs4 import BeautifulSoup
import pandas as pd
import time

chromedriver_autoinstaller.install()


chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

try:
    driver.get("https://www.scrapethissite.com/pages/frames/")
    time.sleep(2)

    frames = driver.find_elements(By.TAG_NAME, 'frame')

    data_list = []

    for index, frame in enumerate(frames):
        try:
            driver.switch_to.frame(index)
            time.sleep(1)
            try:
                turtles_link = driver.find_element(By.LINK_TEXT, 'Turtles')
                turtles_link.click()
                time.sleep(2)
            except NoSuchElementException:
                print(f"'Turtles' link not found in frame {index}.")
                driver.switch_to.default_content()
                continue
            html = driver.page_source

            soup = BeautifulSoup(html, 'html.parser')

            turtle_cards = soup.find_all('div', class_='col-md-4 turtle-family-card')

            for card in turtle_cards:
                family_name = card.find('h4').get_text(strip=True)
                description = card.find('p').get_text(strip=True)
                data_list.append({
                    'frame_index': index,
                    'family_name': family_name,
                    'description': description
                })

            driver.switch_to.default_content()

        except NoSuchFrameException:
            print(f"Frame {index} does not exist.")
            driver.switch_to.default_content()

    df = pd.DataFrame(data_list)

    df.to_csv('scraped_frames_data.csv', index=False)
    print("Data has been saved to 'scraped_frames_data.csv'.")

finally:
    driver.quit()

Data has been saved to 'scraped_frames_data.csv'.
